<a href="https://colab.research.google.com/github/xcsengody/DP/blob/master/xcsengody.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#from google.colab import files
#dataset.to_csv('dataset.csv') 

In [0]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
root='/content/drive/My Drive'

In [0]:
#!pip -q install folium==0.2.1;
#!pip -q install ip2geotools;
!pip -q install geoip2;

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import re
import os
import sys
import seaborn as sns
import pylab as pl
import geoip2.database
from scipy import stats
from pylab import rcParams

In [0]:
pd.set_option('display.max_rows', 500);
rcParams['figure.figsize'] = 15, 8;

In [0]:
path = root+'/DP/GeoLite2-Country.mmdb';
reader=geoip2.database.Reader(path);

In [0]:
path = root+"/DP/Dataset/NUSW-NB15_features.csv";
df = pd.read_csv(path, delimiter=',', encoding='unicode_escape', low_memory=False, skipinitialspace=True, skip_blank_lines=True, verbose=True);

In [0]:
features = df.Name

In [0]:
path = root+"/DP/Dataset/UNSW-NB15_1.csv";
dataset = pd.read_csv(path, delimiter=',', names=features, encoding='utf-8', low_memory=False, skipinitialspace=True, skip_blank_lines=True, verbose=True);

In [0]:
path = root+"/DP/Dataset/dataset.csv";
#dataset = pd.read_csv(path, index=False, delimiter=',', encoding='utf-8', low_memory=False, skipinitialspace=True, skip_blank_lines=True, verbose=True);
del dataset["Unnamed: 0"]

In [0]:
print("Number of rows: {}\nNumber of features: {}".format(dataset.shape[0],dataset.shape[1]))

In [0]:
print("NaN columns {}".format(dataset.columns[dataset.isna().any()].tolist()))

In [0]:
dataset.attack_cat = dataset.attack_cat.fillna('Normal')

In [0]:
dataset = dataset[dataset.state.apply(lambda x: x != 'no')]

In [0]:
dataset.service = dataset.service.replace(to_replace ="-", value ="NUSER");

In [0]:
dataset = dataset[dataset.sport.apply(lambda x: x != '-')]

In [0]:
dataset = dataset[dataset.dsport.apply(lambda x: x != '-')]

In [0]:
dataset.sport = dataset.sport.apply(lambda x: int(x,16) if re.search('0x.*', x) else x)

In [0]:
dataset.dsport = dataset.dsport.apply(lambda x: int(x,16) if re.search('0x.*', x) else x)

In [0]:
dataset = dataset[dataset.sport.apply(lambda x: int(x) <= 65535)]
dataset = dataset[dataset.dsport.apply(lambda x: int(x) <= 65535)]

In [0]:
dataset = dataset[dataset.dur.apply(lambda x: float(x) < 8000)]

In [0]:
dataset.sport.astype(int).max()

In [0]:
dataset.attack_cat.value_counts()

In [0]:
dataset.state.value_counts()

In [0]:
dataset.service.value_counts()

In [0]:
dataset.shape

In [0]:
dataset.head()

In [0]:
dataset[dataset.sloss].columns.value

In [0]:
plt.figure()
fig,ax = plt.subplots()

x=dataset.index.astype(int);
y=dataset.sloss;
mean=[np.mean(y)]*len(x);

data_line = ax.scatter(x, y, label='Data', marker='o', color="lightblue");
mean = ax.plot(x, mean, label='Mean', linestyle='--', color="red");
legend = ax.legend(loc='upper right');
plt.title("sloss");

plt.show()

In [0]:
test=dataset[dataset.sttl.apply(lambda x: x>30 and x<50)]
print(test.shape)
print(test[test.Label.apply(lambda x: x==0)].shape)
print(test[test.Label.apply(lambda x: x==1)].shape)

In [0]:
#sbytes,dbytes and sloss,dloss are in corelation (probably - check it out). All the records are attacs and due to that can not be removed.

#Data normalization should be applied to sttl, dttl
#

In [0]:
plt.figure()
plt.scatter(dataset.index.astype(int), dataset.sttl, color='green')
plt.show()

In [0]:
plt.figure()
plt.scatter(dataset.index.astype(int), dataset.dttl, color='green', marker='o')
plt.show()

In [0]:
#corrmat = dataset.corr()
#top_corr_features = corrmat.index
#pl.figure(figsize=(30,30))
#g=sns.heatmap(dataset[top_corr_features].corr(method='pearson', min_periods=1),annot=True,cmap="RdYlGn")

In [0]:
#dataset.proto.value_counts().plot(kind='bar', title='Histogram protokolov')

#counts = dataset.proto.value_counts()
#all_values = dataset.shape[0]
#perc = counts / all_values
#print(perc)

In [0]:
def portType(x):
  #Ports range:
  #Well known: 0-1023 (1)
  #Registered: 1024-49151 (2)
  #Private: 49152-65535 (3)
  if x >= 0 and x < 1024:
    return 1;
  elif x >= 1024 and x < 49152:
    return 2;
  elif x >= 49152 and x < 65536:
    return 3;

def isPrivateIP(x):
  #192.168.0.0 - 192.168.255.255
  #172.16.0.0 - 172.31.255.255
  #10.0.0.0 - 10.255.255.255
  if (re.search('192\.168\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])',x)) or (re.search('172\.16|31\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])',x)) or (re.search('10\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])',x)) or (re.search('127\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])',x)):
    return True
  else:
    return False

def isLocalhostIP(x):
  #127.0.0.0 - 10.255.255.255
  if (re.search('127\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])',x)):
    return True
  else:
    return False

def isMulticastIP(x):
  if (re.search('22[4-9]|23[0-9]\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])\.(25[0-5]|2[0-4][0-9]|1[0-9][0-9]|[1-9]?[0-9])',x)):
    return True
  else:
    return False

def getIPLocation(ip):
  try:
    location=reader.country(ip);
    return location.country.names['en']+" ("+location.country.iso_code+")";
  except:
    if isPrivateIP(ip):
      return "Private";
    elif isLocalhostIP(ip):
      return "Localhost";
    elif isMulticastIP(ip):
      return "Multicast"; 

def closeReader():
  reader.close();

In [0]:
#dataset.insert(1, "srciploc","")
#dataset.insert(4, "dstiploc","")

#dataset.srciploc=dataset.srcip.apply(lambda x: getIPLocation(x));
#dataset.dstiploc=dataset.dstip.apply(lambda x: getIPLocation(x));

#dataset=dataset.reset_index();
#dataset=dataset.set_index('index');

#attack_traffic=dataset[dataset.Label.apply(lambda x: x==1)]
#normal_traffic=dataset[dataset.Label.apply(lambda x: x==0)]

In [0]:
#attack_traffic.srciploc=attack_traffic.srcip.apply(lambda x: getIPLocation(x));
#attack_traffic.dstiploc=attack_traffic.dstip.apply(lambda x: getIPLocation(x));
#normal_traffic.srciploc=normal_traffic.srcip.apply(lambda x: getIPLocation(x));
#normal_traffic.dstiploc=normal_traffic.dstip.apply(lambda x: getIPLocation(x));
#closeReader();

In [0]:
dataset.groupby(['attack_cat']).size().reset_index(name='counts').sort_values(by='counts', ascending=False)

In [0]:
dataset.shape[1]

ohe_srcip = pd.get_dummies(dataset.srcip,prefix='scrip')
dataset = pd.concat([dataset, ohe_srcip], axis=1)
#dataset = dataset.drop(columns=['protocol_type'])

In [0]:
dataset.shape[1]